
# Propagation of `max_row`: Seed Maximization + BFS

This notebook demonstrates **why we (1) maximize the initial seed value and (2) then propagate via BFS**,
and **why 8-directional adjacency is needed** when exit-to-exit connections can touch diagonally.



## Concept Summary

- We maintain `max_row[g]`: the *maximum reachable row* for a golem/component.
- **Step 1 (Seed Maximization):** If the current golem's exit touches a neighbor's **body**, inherit that neighbor's `max_row` as the initial **seed** (i.e., start as large as possible).
- **Step 2 (BFS Propagation):** Traverse **exit-to-exit** connections (which may include diagonals), and propagate the value to the whole connected component.

Why this two-step design?
- **Accuracy:** If a newly placed golem is already adjacent to a fully computed component, it should **immediately** inherit that component's max value.
- **Efficiency & Simplicity:** With a large seed, a single BFS pass (enqueue only on improvement) is enough. Without it, you may need repeated re-queuing or you might fail to update at all.



## Example 1 — Without Seeding, Propagation Can Fail

Graph: `A — B` (exit-to-exit edge).  
Suppose `max_row[B] = 10` (already computed from earlier steps) and the newly placed `A` starts from its baseline `5`.

We use the **common BFS rule**: only enqueue a neighbor when its value **increases**.


In [1]:

from collections import deque

def bfs_update_only_on_improvement(graph, max_row, start):
    """
    Standard pattern:
    - Start with current max_row values.
    - Enqueue 'start' only.
    - When visiting u -> v, if max_row[v] < max_row[u], update and enqueue v.
    """
    q = deque([start])
    visited_pops = 0
    while q:
        u = q.popleft()
        visited_pops += 1
        for v in graph.get(u, []):
            if max_row[v] < max_row[u]:
                max_row[v] = max_row[u]
                q.append(v)
    return max_row, visited_pops

# Graph A—B
graph = {"A": ["B"], "B": ["A"]}

# Case: B already has 10, A starts at 5 (no seeding)
max_row = {"A": 5, "B": 10}
res, pops = bfs_update_only_on_improvement(graph, max_row, start="A")
res, pops


({'A': 5, 'B': 10}, 1)


**Result:** `A` remains 5, `B` remains 10.  
Propagation **never happens**, because from `A`'s perspective, `10 < 5` is false — there is no improvement to push to `B`, so the traversal ends immediately.

**Correct behavior** should make `A` inherit `10` (since A is connected to B). We fix this by **seeding**.


In [ ]:

# Seeding: initialize A with the max of its own baseline and neighbor's value (here, 10)
max_row = {"A": 10, "B": 10}  # after Step 1 seeding
res, pops = bfs_update_only_on_improvement(graph, max_row, start="A")
res, pops



Now we start with `A=10`. One BFS pass is enough; values are already consistent.



## Example 2 — Forcing Neighbor Queueing Works but Causes Extra Work

Another approach: **enqueue neighbors even without improvement**.  
This can eventually propagate the larger value *backwards*, but it introduces **redundant re-queueing** and more pops, which hurts performance on large graphs.


In [ ]:

from collections import deque

def bfs_force_enqueue_neighbors(graph, max_row, start):
    """
    Non-standard pattern:
    - Always enqueue neighbors (even if there is no improvement yet).
    - This ensures eventual propagation but increases work.
    """
    q = deque([start])
    visited_pops = 0
    enq_edges = 0
    while q:
        u = q.popleft()
        visited_pops += 1
        for v in graph.get(u, []):
            enq_edges += 1
            # Enqueue regardless
            q.append(v)
            # If there is an improvement, apply it
            if max_row[v] < max_row[u]:
                max_row[v] = max_row[u]
    return max_row, visited_pops, enq_edges

graph = {"A": ["B"], "B": ["A"]}
max_row = {"A": 5, "B": 10}  # no seeding
res, pops, enq = bfs_force_enqueue_neighbors(graph, max_row, start="A")
res, pops, enq



We eventually get `A=10`, but notice the **extra pops/enqueues**.
On bigger components with cycles, this overhead grows significantly.
In contrast, **seeding** followed by the standard "enqueue on improvement" BFS finishes in **one clean pass**.



## Example 3 — Why 8-Neighbor (Diagonal) Adjacency?

Exit cells can be **diagonally touching**. If we only check 4 neighbors (up/down/left/right),
we miss such connections and split a true component into two.


In [ ]:

# Two exits that touch diagonally
exit_A = (5, 5)
exit_B = (4, 6)

def neighbors_4(y, x):
    return [(y-1,x), (y+1,x), (y,x-1), (y,x+1)]

def neighbors_8(y, x):
    return [
        (y-1,x), (y+1,x), (y,x-1), (y,x+1),
        (y-1,x-1), (y-1,x+1), (y+1,x-1), (y+1,x+1)
    ]

is_connected_4 = exit_B in neighbors_4(*exit_A)
is_connected_8 = exit_B in neighbors_8(*exit_A)

print("4-neigh sees connection?  ", is_connected_4)
print("8-neigh sees connection?  ", is_connected_8)



As expected, **4-neighborhood fails** while **8-neighborhood succeeds**.
Therefore, Step 2 uses 8-directional checks to avoid missing exit-to-exit connections.



## Takeaways

- **Seed maximization** ensures the start node already has the correct upper bound, so the usual BFS rule
  ("enqueue only on improvement") resolves the whole component in one pass.
- **8-neighborhood** is necessary when exit tiles can touch diagonally, ensuring we don't miss valid connections.
- This design yields both **correctness** and **efficiency**.
